# Download and Inspect on the Dataset

In [19]:
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/"
        "refs/heads/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x79b56ddc4710>)

In [20]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

print("total number of characters:", len(raw_text))
print(raw_text[:500])

total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it'


## Split texts into words

In [21]:
import re

experiment_text = "Hello, World! Let's test this, --sentence."

# the splitting delimiters included in the output
splitted_word_list = re.split(r'([,.:;?_!"()\'\s]|--)', experiment_text)
print(splitted_word_list)


# not included
# splitted_word_list2 = re.split(r'[,.]|\s', experiment_text)
# print(splitted_word_list2)

['Hello', ',', '', ' ', 'World', '!', '', ' ', 'Let', "'", 's', ' ', 'test', ' ', 'this', ',', '', ' ', '', '--', 'sentence', '.', '']


In [22]:
# get rid of white-space
splitted_word_list = [word for word in splitted_word_list if word.strip()]
print(splitted_word_list)

['Hello', ',', 'World', '!', 'Let', "'", 's', 'test', 'this', ',', '--', 'sentence', '.']


Let's apply it to the text

In [23]:
preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', raw_text)
preprocessed = [word for word in preprocessed if word.strip()]
print(preprocessed[:50])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


## Convert tokens into token IDs

In [24]:
set_of_all_words = sorted(set(preprocessed))
print(set_of_all_words)

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed', 'Devonshire', 'Don', 'Dubarry', 'Emperors', 'Florence', 'For', 'Gallery', 'Gideon', 'Gisburn', 'Gisburns', 'Grafton', 'Greek', 'Grindle', 'Grindles', 'HAD', 'Had', 'Hang', 'Has', 'He', 'Her', 'Hermia', 'His', 'How', 'I', 'If', 'In', 'It', 'Jack', 'Jove', 'Just', 'Lord', 'Made', 'Miss', 'Money', 'Monte', 'Moon-dancers', 'Mr', 'Mrs', 'My', 'Never', 'No', 'Now', 'Nutley', 'Of', 'Oh', 'On', 'Once', 'Only', 'Or', 'Perhaps', 'Poor', 'Professional', 'Renaissance', 'Rickham', 'Riviera', 'Rome', 'Russian', 'Sevres', 'She', 'Stroud', 'Strouds', 'Suddenly', 'That', 'The', 'Then', 'There', 'They', 'This', 'Those', 'Though', 'Thwing', 'Thwings', 'To', 'Usually', 'Venetian', 'Victor', 'Was', 'We', 'Well', 'What', 'When', 'Why', 'Yes', 'You', '_', 'a', 'abdication', 'able', 'about', 'above',

In [25]:
vocab_size = len(set_of_all_words)
print(vocab_size)

1130


In [26]:
# print some vocab
vocab = {token:integer for integer, token in enumerate(set_of_all_words)}

for i, token in enumerate(vocab.items()):
  print(token)
  if i == 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Implement a Complete Simple Tokenizer

In [27]:
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.word_to_id = vocab
    self.id_to_word = {id:word for word, id in vocab.items()}

  # word --> id (tokenize)
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', text)
    preprocessed = [word for word in preprocessed if word.strip()]
    id_list = [self.word_to_id[word] for word in preprocessed]
    return id_list

  # id --> word (detokenize)
  def decode(self, id_list):
    word_list = [self.id_to_word[id] for id in id_list]
    text_with_white_space = " ".join(word_list)
    # remove space before punctuations
    text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text_with_white_space)
    return text

In [28]:
simple_tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""

id_list = simple_tokenizer.encode(text)
print(id_list)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [29]:
print(simple_tokenizer.decode(id_list))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


Our simple tokenizer works, but it cannot work on the data out of the text.

## Adding Special Context Tokens

In [30]:
list_of_all_words = sorted(list(set(preprocessed)))
list_of_all_words.extend(["<|unk|>", "<|endoftext|>"])
vocab = {word:id for id, word in enumerate(list_of_all_words)}

print(len(vocab.items()))

for i, word in enumerate(list(vocab.items())[-10:]):
  print(word)

1132
('year', 1122)
('years', 1123)
('yellow', 1124)
('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


## Implement a Simple Tokenizer That Handles Unknown Words

In [36]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.word_to_id = vocab
    self.id_to_word = {id:word for word, id in vocab.items()}

  # word --> id (tokenizer)
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', text)
    preprocessed = [word for word in preprocessed if word.strip()]
    # handle unknown words
    word_list = [word if word in self.word_to_id
              else "<|unk|>" for word in preprocessed]
    id_list = [self.word_to_id[word] for word in word_list]
    return id_list

  # id --> word (detokenizer)
  def decode(self, id_list):
    word_list = [self.id_to_word[id] for id in id_list]
    text_with_white_space = " ".join(word_list)
    # remove space before punctuations
    text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text_with_white_space)
    return text

Let's test our new tokenizer:

In [37]:
text1 = """"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus"""
text2 = """Happy Hunger Games, and may the odds be ever in your favor."""

text = " <|endoftext|> ".join((text1, text2))
print(text)

"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [38]:
tokenizerv2 = SimpleTokenizerV2(vocab)
print(tokenizerv2.encode(text))

[1, 55, 988, 1130, 722, 1130, 5, 53, 469, 5, 1130, 663, 5, 156, 1130, 1130, 1, 851, 1130, 1130, 1131, 1130, 1130, 1130, 5, 157, 662, 988, 1130, 198, 401, 568, 1128, 1130, 7]


In [39]:
print(tokenizerv2.decode(tokenizerv2.encode(text)))

" In the <|unk|> of <|unk|>, I found, <|unk|> me, an <|unk|> <|unk|>" said <|unk|> <|unk|> <|endoftext|> <|unk|> <|unk|> <|unk|>, and may the <|unk|> be ever in your <|unk|>.


## Byte Pair Encoding (PBE)

To handle unknown words at ease, we tokenize subword instead:

[tiktoken doc](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [50]:
from importlib_metadata import version
import tiktoken

print("tiktoken version: ", version("tiktoken"))
print(tiktoken.encoding_for_model("gpt-3.5-turbo"))
print(tiktoken.encoding_for_model("gpt-3.5-turbo").n_vocab)
print(tiktoken.list_encoding_names())

tiktoken version:  0.9.0
<Encoding 'cl100k_base'>
100277
['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']


In [52]:
bpe_tokenizer = tiktoken.get_encoding("gpt2")
print(bpe_tokenizer.n_vocab)
print(text)

50257
"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [53]:
id_list = bpe_tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(id_list)

[1, 818, 262, 15925, 286, 7374, 11, 314, 1043, 11, 1626, 502, 11, 198, 281, 46038, 3931, 1, 531, 9966, 7298, 385, 220, 50256, 14628, 32367, 5776, 11, 290, 743, 262, 10402, 307, 1683, 287, 534, 2661, 13]


In [54]:
decoded_text = bpe_tokenizer.decode(id_list)
print(decoded_text)

"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [56]:
# let's decode into byte
decoded_text_byte = [bpe_tokenizer.decode_single_token_bytes(token)
                    for token in id_list]
print(decoded_text_byte)

[b'"', b'In', b' the', b' midst', b' of', b' winter', b',', b' I', b' found', b',', b' within', b' me', b',', b'\n', b' an', b' invincible', b' summer', b'"', b' said', b' Albert', b' Cam', b'us', b' ', b'<|endoftext|>', b' Happy', b' Hunger', b' Games', b',', b' and', b' may', b' the', b' odds', b' be', b' ever', b' in', b' your', b' favor', b'.']


# Dataset and DataLoader